In [10]:
import openmc
import openmc.deplete as od
import math
import pandas as pd
import numpy as np
import openmc.mgxs as mgxs

In [11]:
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")

radii = [0.42, 0.45]
fuel.volume = math.pi * radii[0] ** 2

materials = openmc.Materials([fuel, clad, water])

In [12]:
fuel.get_nuclide_atom_densities()

OrderedDict([('U234', ('U234', 8.922411359424316e-06)),
             ('U235', ('U235', 0.000998240191860822)),
             ('U238', ('U238', 0.022192386373095892)),
             ('U236', ('U236', 4.572419549506112e-06)),
             ('O16', ('O16', 0.04639065406771322)),
             ('O17', ('O17', 1.7588724018066158e-05))])

In [13]:
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
pin_univ = openmc.model.pin(pin_surfaces, materials)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=pin_univ, region=bound_box)
geometry = openmc.Geometry([root_cell])

In [14]:
settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50
settings.photon_transport = True

In [15]:
#groups = mgxs.EnergyGroups()
#groups.group_edges = np.array([0.,20.0e6])
#fission = mgxs.FissionXS(domain=fuel, groups=groups, by_nuclide=True)
#capture = mgxs.CaptureXS(domain=fuel, groups=groups,by_nuclide=True)

tally = openmc.Tally()
tally.scores = ['flux', 'heating']

tallies = openmc.Tallies()
tallies.append(tally)

In [16]:
model = openmc.model.Model(geometry, materials, settings, tallies)

In [17]:
model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/ooblack/projects/openmc-depletion-testing/statepoint.50.h5')

In [37]:
#operator = openmc.deplete.Operator(model, "../openmc/tests/chain_simple.xml", normalization_mode='fission-q')
power = 174
#time_steps = [30]
#integrator = openmc.deplete.PredictorIntegrator(operator, time_steps, power, timestep_units='d')

## Get Flux

In [18]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')

tally = sp.get_tally(scores=['flux', 'heating'])

# Close the statepoint file now that we're done getting info
sp.close()

In [26]:
flux = flux_tally.get_slice(scores=['flux'])
heating = flux_tally.get_slice(scores=['heating'])

In [38]:
heating_rate = 1.602e-19 * heating.mean.flatten()[0]
normalization_factor = power / heating_rate

In [43]:
real_flux = flux.mean.flatten()[0] * normalization_factor / (math.pi * radii[1] ** 2)

In [44]:
real_flux

1164719970082145.0